In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from interstellar import Tracks, DenseNetwork, renormalize, normalize
from scipy import stats

In [2]:
name = 'network_10'
network_path = f'../data/networks/{name}'

tracks = 'tracks_test_random'
# test = pd.read_csv(f'/home/ADF/ajl573/Data/interstellar_data/{tracks}/{tracks}_data.csv')
test = pd.read_csv(f'/rds/projects/d/daviesgr-alex-phd/interstellar_data/{tracks}/{tracks}_data.csv')

In [3]:
test.head()

,evol_stage,initial_mass,initial_Yinit,initial_Zinit,initial_feh,initial_MLT,star_age,frac_age,effective_T,luminosity,radius,log_g,surface_he4,surface_Z,star_feh,delta_nu_fit,frac_evol,gradient
0,0,1.150029,0.306814,0.01662,0.132709,2.107285,0.042021,0.010224,6288.6006,1.726106,1.108370,4.409229,0.306146,0.016591,0.131459,123.787140,0.010224,0.000409
1,0,1.150029,0.306814,0.01662,0.132709,2.107285,0.045031,0.010956,6290.8853,1.726764,1.107776,4.409695,0.306048,0.016586,0.131264,123.874344,0.010956,0.000355
2,0,1.150029,0.306814,0.01662,0.132709,2.107285,0.048643,0.011835,6292.6210,1.728088,1.107589,4.409841,0.305930,0.016580,0.131028,123.897390,0.011835,0.000274
3,1,1.150029,0.306814,0.01662,0.132709,2.107285,0.052977,0.012890,6294.0464,1.730105,1.107733,4.409728,0.305786,0.016573,0.130742,123.867320,0.012890,0.000235
4,1,1.150029,0.306814,0.01662,0.132709,2.107285,0.058179,0.014155,6295.2783,1.732630,1.108108,4.409434,0.305615,0.016564,0.130403,123.800160,0.014155,0.000216


## log cols

From these histograms it makes sense to log star age and radius, with everything else to be scaled in the training set.

In [4]:
test['log_star_age'] = np.log10(test['star_age'])

In [5]:
print('Length = ', len(test))
test.head()

Length =  2930919


,evol_stage,initial_mass,initial_Yinit,initial_Zinit,initial_feh,initial_MLT,star_age,frac_age,effective_T,luminosity,radius,log_g,surface_he4,surface_Z,star_feh,delta_nu_fit,frac_evol,gradient,log_star_age
0,0,1.150029,0.306814,0.01662,0.132709,2.107285,0.042021,0.010224,6288.6006,1.726106,1.108370,4.409229,0.306146,0.016591,0.131459,123.787140,0.010224,0.000409,-1.376535
1,0,1.150029,0.306814,0.01662,0.132709,2.107285,0.045031,0.010956,6290.8853,1.726764,1.107776,4.409695,0.306048,0.016586,0.131264,123.874344,0.010956,0.000355,-1.346489
2,0,1.150029,0.306814,0.01662,0.132709,2.107285,0.048643,0.011835,6292.6210,1.728088,1.107589,4.409841,0.305930,0.016580,0.131028,123.897390,0.011835,0.000274,-1.312980
3,1,1.150029,0.306814,0.01662,0.132709,2.107285,0.052977,0.012890,6294.0464,1.730105,1.107733,4.409728,0.305786,0.016573,0.130742,123.867320,0.012890,0.000235,-1.275909
4,1,1.150029,0.306814,0.01662,0.132709,2.107285,0.058179,0.014155,6295.2783,1.732630,1.108108,4.409434,0.305615,0.016564,0.130403,123.800160,0.014155,0.000216,-1.235236


In [6]:
network = DenseNetwork.from_config(f'{network_path}/{name}_config.json')
network.load_model(suffix='best_model')
network.load_history()
network.model.summary()

Model: "network_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 5)]               0         
_________________________________________________________________
dense (Dense)                (None, 128)               768       
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_3 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_4 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_5 (Dense)              (None, 128)               1

In [7]:
factors = pd.read_csv(f'{network_path}/{name}_norm.csv')

In [8]:
factors

,column,norm_column,loc,scale
0,frac_evol,frac_evol_norm,0.865,0.651
1,initial_mass,initial_mass_norm,1.000,0.118
2,initial_MLT,initial_MLT_norm,1.900,0.338
3,initial_Yinit,initial_Yinit_norm,0.280,0.028
4,initial_Zinit,initial_Zinit_norm,0.017,0.011
5,log_star_age,log_star_age_norm,0.790,0.467
6,effective_T,effective_T_norm,5566.772,601.172
7,radius,radius_norm,1.224,0.503
8,delta_nu_fit,delta_nu_fit_norm,100.720,42.582
9,star_feh,star_feh_norm,0.081,0.361


In [9]:
test[factors['norm_column']] = normalize(test[factors['column']], loc=factors['loc'].to_numpy(), scale=factors['scale'].to_numpy())

In [10]:
test.head()

,evol_stage,initial_mass,initial_Yinit,initial_Zinit,initial_feh,initial_MLT,star_age,frac_age,effective_T,luminosity,...,frac_evol_norm,initial_mass_norm,initial_MLT_norm,initial_Yinit_norm,initial_Zinit_norm,log_star_age_norm,effective_T_norm,radius_norm,delta_nu_fit_norm,star_feh_norm
0,0,1.150029,0.306814,0.01662,0.132709,2.107285,0.042021,0.010224,6288.6006,1.726106,...,-1.313020,1.271433,0.613269,0.957646,-0.034573,-4.639261,1.200702,-0.229882,0.541711,0.139776
1,0,1.150029,0.306814,0.01662,0.132709,2.107285,0.045031,0.010956,6290.8853,1.726764,...,-1.311895,1.271433,0.613269,0.957646,-0.034573,-4.574924,1.204503,-0.231062,0.543759,0.139237
2,0,1.150029,0.306814,0.01662,0.132709,2.107285,0.048643,0.011835,6292.6210,1.728088,...,-1.310545,1.271433,0.613269,0.957646,-0.034573,-4.503169,1.207390,-0.231434,0.544300,0.138583
3,1,1.150029,0.306814,0.01662,0.132709,2.107285,0.052977,0.012890,6294.0464,1.730105,...,-1.308925,1.271433,0.613269,0.957646,-0.034573,-4.423789,1.209761,-0.231147,0.543594,0.137791
4,1,1.150029,0.306814,0.01662,0.132709,2.107285,0.058179,0.014155,6295.2783,1.732630,...,-1.306981,1.271433,0.613269,0.957646,-0.034573,-4.336693,1.211810,-0.230402,0.542017,0.136849


## Validation plots

In [11]:
# network.plot_metric(test.sample(20000), metric='absolute_error', save_plot=False);

In [12]:
# network.plot_metric(test.sample(20000), metric='error', save_plot=False);

In [13]:
test_pred = network.model.predict(test[network.x_cols], batch_size=network.batch_size)
test_pred = pd.DataFrame(test_pred, columns=network.y_cols)

In [14]:
test_pred = test_pred.join(Tracks.renormalize(_, test_pred, factors=factors[factors['norm_column'].isin(network.y_cols)]))

In [15]:
log_cols = [col for col in test_pred.columns if 'log' in col]
log_cols

['log_star_age_norm', 'log_star_age']

In [16]:
for col in log_cols:
    test_pred[col[4:]] = 10**test_pred[col]
    if col in test.columns:
        test[col[4:]] = 10**test[col]

In [17]:
pred_cols = {col: f'{col}_pred' for col in test_pred.columns}
pred_cols

{'log_star_age_norm': 'log_star_age_norm_pred',
 'effective_T_norm': 'effective_T_norm_pred',
 'radius_norm': 'radius_norm_pred',
 'delta_nu_fit_norm': 'delta_nu_fit_norm_pred',
 'star_feh_norm': 'star_feh_norm_pred',
 'log_star_age': 'log_star_age_pred',
 'effective_T': 'effective_T_pred',
 'radius': 'radius_pred',
 'delta_nu_fit': 'delta_nu_fit_pred',
 'star_feh': 'star_feh_pred',
 'star_age_norm': 'star_age_norm_pred',
 'star_age': 'star_age_pred'}

In [18]:
test_pred.rename(columns=pred_cols, inplace=True)
test_pred.head()

,log_star_age_norm_pred,effective_T_norm_pred,radius_norm_pred,delta_nu_fit_norm_pred,star_feh_norm_pred,log_star_age_pred,effective_T_pred,radius_pred,delta_nu_fit_pred,star_feh_pred,star_age_norm_pred,star_age_pred
0,-4.634264,1.198878,-0.231424,0.546347,0.138771,-1.374201,6287.503906,1.107594,123.984535,0.131096,0.000023,0.042247
1,-4.573037,1.201293,-0.230923,0.545071,0.138235,-1.345608,6288.955566,1.107846,123.930206,0.130903,0.000027,0.045122
2,-4.503357,1.203826,-0.230338,0.543621,0.137590,-1.313068,6290.478516,1.108140,123.868454,0.130670,0.000031,0.048633
3,-4.425329,1.206396,-0.229660,0.541981,0.136812,-1.276629,6292.023438,1.108481,123.798645,0.130389,0.000038,0.052890
4,-4.338869,1.208910,-0.228869,0.540132,0.135875,-1.236252,6293.534668,1.108879,123.719910,0.130051,0.000046,0.058043


In [19]:
df = test.join(test_pred)
df.head()

,evol_stage,initial_mass,initial_Yinit,initial_Zinit,initial_feh,initial_MLT,star_age,frac_age,effective_T,luminosity,...,radius_norm_pred,delta_nu_fit_norm_pred,star_feh_norm_pred,log_star_age_pred,effective_T_pred,radius_pred,delta_nu_fit_pred,star_feh_pred,star_age_norm_pred,star_age_pred
0,0,1.150029,0.306814,0.01662,0.132709,2.107285,0.042021,0.010224,6288.6006,1.726106,...,-0.231424,0.546347,0.138771,-1.374201,6287.503906,1.107594,123.984535,0.131096,0.000023,0.042247
1,0,1.150029,0.306814,0.01662,0.132709,2.107285,0.045031,0.010956,6290.8853,1.726764,...,-0.230923,0.545071,0.138235,-1.345608,6288.955566,1.107846,123.930206,0.130903,0.000027,0.045122
2,0,1.150029,0.306814,0.01662,0.132709,2.107285,0.048643,0.011835,6292.6210,1.728088,...,-0.230338,0.543621,0.137590,-1.313068,6290.478516,1.108140,123.868454,0.130670,0.000031,0.048633
3,1,1.150029,0.306814,0.01662,0.132709,2.107285,0.052977,0.012890,6294.0464,1.730105,...,-0.229660,0.541981,0.136812,-1.276629,6292.023438,1.108481,123.798645,0.130389,0.000038,0.052890
4,1,1.150029,0.306814,0.01662,0.132709,2.107285,0.058179,0.014155,6295.2783,1.732630,...,-0.228869,0.540132,0.135875,-1.236252,6293.534668,1.108879,123.719910,0.130051,0.000046,0.058043


In [20]:
del(test)
del(test_pred)

In [21]:
err_cols = [f'{col}_err' for col in pred_cols.keys()]
err_cols

['log_star_age_norm_err',
 'effective_T_norm_err',
 'radius_norm_err',
 'delta_nu_fit_norm_err',
 'star_feh_norm_err',
 'log_star_age_err',
 'effective_T_err',
 'radius_err',
 'delta_nu_fit_err',
 'star_feh_err',
 'star_age_norm_err',
 'star_age_err']

In [22]:
df[err_cols] = pd.DataFrame(df[pred_cols.keys()].to_numpy() - df[pred_cols.values()].to_numpy())

In [23]:
df['log_g_pred'] = 4.44 + np.log10(df['initial_mass']) - 2 * np.log10(df['radius_pred'])

In [24]:
df.head()

,evol_stage,initial_mass,initial_Yinit,initial_Zinit,initial_feh,initial_MLT,star_age,frac_age,effective_T,luminosity,...,delta_nu_fit_norm_err,star_feh_norm_err,log_star_age_err,effective_T_err,radius_err,delta_nu_fit_err,star_feh_err,star_age_norm_err,star_age_err,log_g_pred
0,0,1.150029,0.306814,0.01662,0.132709,2.107285,0.042021,0.010224,6288.6006,1.726106,...,-0.004636,0.001005,-0.002334,1.096694,0.000776,-0.197395,0.000363,-2.655841e-07,-0.000226,4.411948
1,0,1.150029,0.306814,0.01662,0.132709,2.107285,0.045031,0.010956,6290.8853,1.726764,...,-0.001312,0.001001,-0.000881,1.929734,-0.000070,-0.055862,0.000361,-1.158398e-07,-0.000091,4.411750
2,0,1.150029,0.306814,0.01662,0.132709,2.107285,0.048643,0.011835,6292.6210,1.728088,...,0.000679,0.000993,0.000088,2.142484,-0.000551,0.028936,0.000358,1.355388e-08,0.000010,4.411520
3,1,1.150029,0.306814,0.01662,0.132709,2.107285,0.052977,0.012890,6294.0464,1.730105,...,0.001613,0.000979,0.000719,2.022963,-0.000748,0.068675,0.000353,1.334276e-07,0.000088,4.411252
4,1,1.150029,0.306814,0.01662,0.132709,2.107285,0.058179,0.014155,6295.2783,1.732630,...,0.001885,0.000974,0.001016,1.743632,-0.000771,0.080250,0.000352,2.301139e-07,0.000136,4.410941


In [25]:
# condition = (np.isclose(df['initial_MLT'], 2.0)) &\
#             (np.isclose(df['initial_Yinit'], 0.27))

In [26]:
# fig, ax = plt.subplots(figsize=(16, 16))

# df[condition].abs().plot(ax=ax, kind='scatter', x='effective_T', y='log_g', 
#         c='star_age_err', cmap='plasma', s=1, alpha=0.5)

# ax.invert_xaxis()
# ax.invert_yaxis()

In [27]:
# fig, ax = plt.subplots(figsize=(16, 16))

# df[condition].abs().plot(ax=ax, kind='scatter', x='effective_T', y='log_g', 
#         c='delta_nu_fit_err', cmap='plasma', s=1, alpha=0.5, vmax=3.5)

# ax.invert_xaxis()
# ax.invert_yaxis()

In [28]:
# fig, ax = plt.subplots(figsize=(16, 16))

# df[condition].abs().plot(ax=ax, kind='scatter', x='effective_T', y='log_g', 
#         c='star_feh_err', cmap='plasma', s=1, alpha=0.5)

# ax.invert_xaxis()
# ax.invert_yaxis()

In [29]:
# fig, ax = plt.subplots(figsize=(16, 16))

# df[condition].abs().plot(ax=ax, kind='scatter', x='effective_T', y='log_g', 
#         c='radius_err', cmap='plasma', s=1, alpha=0.5)

# ax.invert_xaxis()
# ax.invert_yaxis()

In [30]:
# fig, ax = plt.subplots(figsize=(16, 16))

# df[condition].abs().plot(ax=ax, kind='scatter', x='effective_T', y='log_g', 
#         c='effective_T_err', cmap='plasma', s=1, alpha=0.5)

# ax.invert_xaxis()
# ax.invert_yaxis()

In [31]:
def plot_rolling(data, column, suffix='err', 
                 window_width=100000, window_type=None, window_kwargs={}, ax=None, subplot_kwargs={}):
    ''' Plots rolling stats of the error of a given `column` in `data`. 
    The `window_width` is the size of the convolution in rows of `data`.'''
    if ax is None:
        _, ax = plt.subplots(**subplot_kwargs)
    
    err_col = f'{column}_{suffix}'
    data_sort = data.sort_values(column)
    
    rolling_mean = data_sort[err_col].rolling(window_width, win_type=window_type).mean(**window_kwargs)
    rolling_median = data_sort[err_col].rolling(window_width, win_type=window_type).median(**window_kwargs)
#     rolling_std = data_sort[err_col].rolling(window_width, win_type=window_type).std(**window_kwargs)
#     rolling_3 = data_sort[err_col].rolling(window_width, win_type=window_type).quantile(.025, **window_kwargs)
    rolling_16 = data_sort[err_col].rolling(window_width, win_type=window_type).quantile(.16, **window_kwargs)
    rolling_84 = data_sort[err_col].rolling(window_width, win_type=window_type).quantile(.84, **window_kwargs)
#     rolling_98 = data_sort[err_col].rolling(window_width, win_type=window_type).quantile(.975, **window_kwargs)

    ax.plot(data_sort[column], rolling_mean, 'k', alpha=0.8, label='mean')
    ax.plot(data_sort[column], rolling_median, 'k--', alpha=0.8, label='median')
#     ax.fill_between(data_sort[column], rolling_mean+rolling_std, rolling_mean-rolling_std, 
#                     facecolor='grey', alpha=0.4, label='rolling std')
#     ax.fill_between(data_sort[column], rolling_98, rolling_3, 
#                     facecolor='grey', alpha=0.2, label='95%')
    ax.fill_between(data_sort[column], rolling_84, rolling_16, 
                    facecolor='grey', alpha=0.5, label='68%')

    
    ax.set_xlabel(column)
    ax.set_ylabel(err_col)
#     ax.legend();
    return ax

In [32]:
df.columns

Index(['evol_stage', 'initial_mass', 'initial_Yinit', 'initial_Zinit',
       'initial_feh', 'initial_MLT', 'star_age', 'frac_age', 'effective_T',
       'luminosity', 'radius', 'log_g', 'surface_he4', 'surface_Z', 'star_feh',
       'delta_nu_fit', 'frac_evol', 'gradient', 'log_star_age',
       'frac_evol_norm', 'initial_mass_norm', 'initial_MLT_norm',
       'initial_Yinit_norm', 'initial_Zinit_norm', 'log_star_age_norm',
       'effective_T_norm', 'radius_norm', 'delta_nu_fit_norm', 'star_feh_norm',
       'star_age_norm', 'log_star_age_norm_pred', 'effective_T_norm_pred',
       'radius_norm_pred', 'delta_nu_fit_norm_pred', 'star_feh_norm_pred',
       'log_star_age_pred', 'effective_T_pred', 'radius_pred',
       'delta_nu_fit_pred', 'star_feh_pred', 'star_age_norm_pred',
       'star_age_pred', 'log_star_age_norm_err', 'effective_T_norm_err',
       'radius_norm_err', 'delta_nu_fit_norm_err', 'star_feh_norm_err',
       'log_star_age_err', 'effective_T_err', 'radius_err', 'delta

In [33]:
solar_teff = 5777.
df['luminosity_pred'] = df['radius_pred']**2 * (df['effective_T_pred']/solar_teff)**4
df['luminosity_err'] = df['luminosity'] - df['luminosity_pred']

In [34]:
# df_sample = df.sample(50000)
df_sample = df[df['star_age']<15].copy()

In [35]:
del(df)

In [36]:
col = 'delta_nu_fit'
df_sample[f'{col}_rel_err'] = abs(df_sample[f'{col}_err'] / df_sample[col])

In [37]:
col = 'delta_nu_fit'
df_sample[f'{col}_abs_rel_err'] = abs(df_sample[f'{col}_rel_err'])

In [65]:
df_sample[err_cols]

,star_age_per_err,effective_T_err,radius_per_err,luminosity_per_err,delta_nu_fit_err,star_feh_err
0,-0.538825,1.096694,0.070018,0.276657,-0.197395,0.000363
1,-0.203022,1.929734,-0.006346,0.177073,-0.055862,0.000361
2,0.020161,2.142484,-0.049745,0.103870,0.028936,0.000358
3,0.165486,2.022963,-0.067512,0.060773,0.068675,0.000353
4,0.233672,1.743632,-0.069590,0.038869,0.080250,0.000352
...,...,...,...,...,...,...
2930914,-0.020084,0.657752,-0.103760,-0.087895,0.076744,0.000040
2930915,-0.019935,0.673783,-0.101846,-0.082776,0.076157,0.000028
2930916,-0.019725,0.750047,-0.105576,-0.084139,0.079014,0.000131
2930917,-0.019503,0.757691,-0.102731,-0.077807,0.078048,0.000123


In [38]:
columns = [
    'star_age', 'effective_T', 'radius', 
    'luminosity', 'delta_nu_fit', 'star_feh'
]

suffixes = [
    'per_err', 'err', 'per_err', 
    'per_err', 'err', 'err'
]

labels = {
    'star_age': [r'\tau', r'(\mathrm{Gyr})'],
    'effective_T': [r'T_\mathrm{eff}' ,r'(\mathrm{K})'],
    'radius': [r'R', r'(\mathrm{R}_\odot)'],
    'luminosity': [r'L', r'(\mathrm{L}_\odot)'],
    'delta_nu_fit': [r'\Delta\nu', r'(\mathrm{\mu Hz})'],
    'star_feh': [r'[\mathrm{M}/\mathrm{H}]_\mathrm{surf}', r'(\mathrm{dex})'],
}

In [39]:
# plt.style.use('seaborn-paper')
# fig, axes = plt.subplots(3, 2, figsize=(8, 9), dpi=300,
#                         gridspec_kw={'wspace': 0.3, 'hspace': 0.3})

# columns = [
#     'star_age', 'effective_T', 'radius', 
#     'luminosity', 'delta_nu_fit', 'star_feh'
# ]

# labels = {
#     'star_age': [r'\tau', r'(\mathrm{Gyr})'],
#     'effective_T': [r'T_\mathrm{eff}' ,r'(\mathrm{K})'],
#     'radius': [r'R', r'(\mathrm{R}_\odot)'],
#     'luminosity': [r'L', r'(\mathrm{L}_\odot)'],
#     'delta_nu_fit': [r'\Delta\nu', r'(\mathrm{\mu Hz})'],
#     'star_feh': [r'[\mathrm{M}/\mathrm{H}]_\mathrm{surf}', r'(\mathrm{dex})'],
# }
# plot_legend = True
# for ax, col in zip(axes.flatten(), columns):
#     plot_rolling(df_sample, col, ax=ax)
#     if '_' in labels[col][0]:
#         ax.set_xlabel('$'+labels[col][0]+r'^\mathrm{true}\,'+labels[col][1]+'$')
#         ax.set_ylabel('$'+labels[col][0]+r'^\mathrm{true} - '+\
#                           labels[col][0]+r'^\mathrm{pred}\,'+labels[col][1]+'$')
#     else:
#         ax.set_xlabel('$'+labels[col][0]+r'_\mathrm{true}\,'+labels[col][1]+'$')
#         ax.set_ylabel('$'+labels[col][0]+r'_\mathrm{true} - '+\
#                           labels[col][0]+r'_\mathrm{pred}\,'+labels[col][1]+'$')
#     if plot_legend:
#         ax.legend()
#         plot_legend = False

# fig.savefig('../paper/figures/validation.png', bbox_inches='tight')

In [40]:
# cond = test['star_age'] < 15.0
# scores = network.scores(test[cond], metric='error', predict_kw={'batch_size': network.batch_size})
# scores.head()

In [41]:
# scale_scores = factors.loc[5:, 'scale'].to_numpy() * scores
# scale_scores

In [42]:
# log_cols = [col for col in scale_scores.columns if 'log' in col]
# log_cols

In [43]:
# scale_scores[[col[4:] for col in log_cols]] = scale_scores[log_cols] * np.log(10.0)

In [44]:
# scale_scores.loc['std'].round(5)

Note: here, star_age_norm is a percent error and the rest are absolute

In [61]:
def plot_err_dist(data, column, x_range=None, ax=None, subplot_kwargs={}, n_sig=4):
#     mle_norm = stats.norm.fit(data[column])
    mle_norm = (np.median(data[column]), stats.median_absolute_deviation(data[column]))
#     mle_t = stats.t.fit(data[column])
    mle_t = None
    
    if ax is None:
        _, ax = plt.subplots(**subplot_kwargs)
    if x_range is None:
        x_range = [mle_norm[0]-n_sig*mle_norm[1], mle_norm[0]+n_sig*mle_norm[1]]
    x = np.linspace(*x_range, 1000)
    pdf_norm = stats.norm.pdf(x, *mle_norm)
#     pdf_t = stats.t.pdf(x, *mle_t)
    
    sns.distplot(data.loc[data[column].between(*x_range), column], label='KDE', vertical=True, ax=ax)
#     ax = data.plot(kind='hist', y=column, bins=100, range=x_range, density=True, facecolor='grey')
#     ax.plot(pdf_t, x, label="student's t")
    ax.plot(pdf_norm, x, label=r'$\mathcal{N}(\mu_\frac{1}{2}, \sigma_\mathrm{MAD})$')

#     ax.set_ylim(x_range)
    col = column[:-4]
#     ax.set_ylabel('$'+labels[col][0]+r'_\mathrm{true} - '+\
#                       labels[col][0]+r'_\mathrm{pred}\,'+labels[col][1]+'$')
    ax.set_xlabel('density')
#     ax.legend()
    return ax, mle_norm, mle_t

In [62]:
for col in columns:
    df_sample[f'{col}_rel_err'] = df_sample[f'{col}_err'] / df_sample[col]
    df_sample[f'{col}_per_err'] = 100 * df_sample[f'{col}_rel_err']

In [63]:
len(df_sample)

2584908

In [64]:
%matplotlib inline
fig, axes = plt.subplots(6, 2, figsize=(5, 15), sharey='row',
                        gridspec_kw={'wspace': 0.05, 'hspace': 0.35, 'width_ratios': [1.5, 1]})

plot_dist = False
plot_legend = True
n_sig = 5

dd = df_sample
window_width=100000

for ax, axdist, col, suffix in zip(axes[:, 0], axes[:, 1], columns, suffixes):
    if plot_dist:
        data = (dd[columns] - dd[columns].min()) / \
            (dd[columns].max() - dd[columns].min())
        for dcol in columns:
            sns.distplot(data[dcol], ax=ax, label='$'+labels[dcol][0]+'$')
        ax.legend(bbox_to_anchor=(1.01, 0.5), loc='center left')
        ax.set_xlabel('min-max scaled')
        ax.set_ylabel('KDE')
        plot_dist = False
        
        axdist.remove()
        continue
    
#     if col == 'star_feh':
#         suffix = 'err'
#     else:
#         suffix = 'rel_err'
    plot_rolling(dd, col, suffix=suffix, ax=ax, window_width=window_width)
    _, mle_norm, mle_t = plot_err_dist(dd, '_'.join([col, suffix]), ax=axdist, n_sig=n_sig)
    axdist.set_ylabel('')
    ax.set_ylim(0.0-n_sig*mle_norm[1], 0.0+n_sig*mle_norm[1])
#     if '_' in labels[col][0]:
#         script = '^'
#     else:
#         script = '_'
    
    ax.set_xlabel('$'+labels[col][0]+r'\,'+labels[col][1]+'$')
    
    if suffix == 'err':
        ylabel = '$\delta '+labels[col][0]+'\,'+labels[col][1]+'$'
    else:
        ylabel = '$\delta '+labels[col][0]+'/'+labels[col][0]
        if suffix == 'per_err':
            ylabel += r'\,(\%)$'
        else:
            ylabel += '$'
    
    ax.set_ylabel(ylabel)
    if plot_legend:
        ax.legend(loc='upper left', ncol=2)
        axdist.legend(loc='upper right')
        plot_legend = False

/rds/homes/a/ajl573/.local/lib/python3.7/site-packages/seaborn/distributions.py:198: RuntimeWarning: Mean of empty slice.
  line, = ax.plot(0, a.mean())


ValueError: Axis limits cannot be NaN or Inf

In [79]:
fig.savefig('../paper/figures/test_random.png', dpi=300, bbox_inches='tight')

In [80]:
plt.close(fig)

In [81]:
df = pd.read_csv('../modelling/final_models/stars_results/DR14_ASPC/stars/results.csv')

In [82]:
df['lum_sd'].min()

0.019612183794379238

In [48]:
print('Length = ', len(df_sample))

Length =  2584908


In [49]:
err_cols = ['_'.join([col, suffix]) for col, suffix in zip(columns, suffixes)]
index = []
for label, suffix in zip(labels.values(), suffixes):
    if suffix == 'err':
        ylabel = '$\delta '+label[0]+'\,'+label[1]+'$'
    else:
        ylabel = '$\delta '+label[0]+'/'+label[0]
        if suffix == 'per_err':
            ylabel += r'\,(\%)$'
        else:
            ylabel += '$'
    index.append(ylabel)
# index = ['$'+label[0]+r'^\mathrm{true} - '+\
#              label[0]+r'^\mathrm{pred}\,'+\
#              label[1]+'$' for label in labels.values()]
summary = pd.DataFrame()
summary[r'\textbf{Error}'] = index
summary

,\textbf{Error}
0,"$\delta \tau/\tau\,(\%)$"
1,"$\delta T_\mathrm{eff}\,(\mathrm{K})$"
2,"$\delta R/R\,(\%)$"
3,"$\delta L/L\,(\%)$"
4,"$\delta \Delta\nu\,(\mathrm{\mu Hz})$"
5,$\delta [\mathrm{M}/\mathrm{H}]_\mathrm{surf}\...


In [55]:
stats.median_absolute_deviation(

nan

In [50]:
summary[r'$\mu_{1/2}$'] = df_sample[err_cols].median().to_numpy()
summary[r'$\sigma_\mathrm{MAD}$'] = stats.median_absolute_deviation(df_sample[err_cols].to_numpy())
# summary[r'$2.5\%$'] = df_sample[err_cols].quantile(.025).to_numpy()
# summary[r'$16\%$'] = df_sample[err_cols].quantile(.16).to_numpy()
# summary[r'$50\%$'] = df_sample[err_cols].quantile(.5).to_numpy()
# summary[r'$84\%$'] = df_sample[err_cols].quantile(.84).to_numpy()
# summary[r'$97.5\%$'] = df_sample[err_cols].quantile(.975).to_numpy()

In [51]:
summary

,\textbf{Error},$\mu_{1/2}$,$\sigma_\mathrm{MAD}$
0,"$\delta \tau/\tau\,(\%)$",-0.011560,NaN
1,"$\delta T_\mathrm{eff}\,(\mathrm{K})$",0.069766,NaN
2,"$\delta R/R\,(\%)$",-0.011221,NaN
3,"$\delta L/L\,(\%)$",0.053054,NaN
4,"$\delta \Delta\nu\,(\mathrm{\mu Hz})$",0.000217,NaN
5,$\delta [\mathrm{M}/\mathrm{H}]_\mathrm{surf}\...,0.000105,NaN


In [100]:
summary.to_csv('test_random.csv')

In [103]:
with open('../paper/test_random.tex', 'w') as file:
    with pd.option_context("max_colwidth", 1000):
        s = summary.to_latex(na_rep='---', escape=False, index=False,
                             column_format='crr',
                             float_format='{:.3f}'.format)
    file.write(s)